# Machine Learning with Python: Part 2

In this tutorial, we will work on a dataset about bank loans. We will use the data to predict whether a loan will be given to an applicant. 

We will learn about the following topics in this tutorial:
- Encoding categorical variables
- Cross-validation
- Hyperparameter tuning
- Feature selection

Let's start by loading the dataset and taking a look at the data.

In [3]:
import pandas as pd

df = pd.read_csv("loans.csv")
df.head()

,gender,married,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amount,term,credit_history,area,loan_given
0,Male,No,0,Graduate,No,584900,0.0,15000000,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,458300,150800.0,12800000,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,300000,0.0,6600000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,258300,235800.0,12000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,600000,0.0,14100000,360.0,1.0,Urban,Y


The dataset contains information about loan applicants, including their gender, marital status, number of dependents, education level, employment status, income, loan amount, loan term, credit history, area, and whether a loan was given or not.

The column `loan_given` is our target variable (the variable we want to predict). The other columns are our features (the variables we use to predict the target variable).

# Cleaning the Data

Before we can use the data for machine learning, we need to clean it. Let's handle missing values first.

In [4]:
df.isna().sum() / len(df) * 100

gender                2.117264
married               0.488599
dependents            2.442997
education             0.000000
self_employed         5.211726
applicant_income      0.000000
coapplicant_income    0.000000
loan_amount           0.000000
term                  2.280130
credit_history        8.143322
area                  0.000000
loan_given            0.000000
dtype: float64

The column with the highest number of missing values is `credit_history`. Still, it is only missing 8.14% of the values, so we can drop the rows with missing values.

In [5]:
df.dropna(inplace=True)
df.isna().sum() / len(df) * 100

gender                0.0
married               0.0
dependents            0.0
education             0.0
self_employed         0.0
applicant_income      0.0
coapplicant_income    0.0
loan_amount           0.0
term                  0.0
credit_history        0.0
area                  0.0
loan_given            0.0
dtype: float64

# Encoding Categorical Variables

So far, we have only worked with numerical variables. However, many datasets contain categorical variables. Categorical variables are variables that have a limited number of possible values. For example, the variable `self_employed` in our dataset has two possible values: `Yes` and `No`. The variable `area` has three possible values: `Urban`, `Rural`, and `Semiurban`.

Machine learning algorithms cannot work with categorical variables directly. For the computer, the values `Yes` and `No` are just strings. Therefore, we need to convert categorical variables into numerical variables. This process is called **encoding**.

There are two main ways to encode categorical variables: **label encoding** and **one-hot encoding**.

## Label Encoding

Label encoding is the process of converting each value of a categorical variable into a number. For example, we can convert the values `Yes` and `No` into the numbers `1` and `0`.

Label encoding is suitable for ordinal variables where there is a meaningful order or hierarchy among the categories. For instance, consider a variable `size` with categories `Small`, `Medium`, and `Large`. We can convert these categories into the numbers `0`, `1`, and `2` because there is a meaningful order among them.

However, label encoding is not suitable for nominal variables where there is no meaningful order among the categories. For instance, consider a variable `colour` with categories `Red`, `Green`, and `Blue`. We cannot convert these categories into the numbers `0`, `1`, and `2` because we cannot say that one colour is greater than another.

**For our dataset, label encoding is not suitable because our categorical variables are nominal variables**. Nevertheless, let's see how we can do label encoding using in Python using the `LabelEncoder` class from the `sklearn.preprocessing` module.

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
area = le.fit_transform(df["area"])
display(area)

`LabelEncoder` has two main methods: `fit` and `transform`. The `fit` method learns the mapping between the categories and the numbers. The `transform` method applies the mapping to the categories and converts them into numbers.

`fit_transform` is a shortcut that combines `fit` and `transform` into a single method.

### Task: encode the variables  `married`, `education`, and `self_employed` using label encoding.

Challenge: Try to encode all of them in a single line of code.

In [ ]:
df[["married", "education", "self_employed"]].apply(le.fit_transform)

## One-Hot Encoding

One-hot encoding is the process of converting each value of a categorical variable into a vector of zeros and ones. For example, we can convert the values `Yes` and `No` into the vectors `[1, 0]` and `[0, 1]`.

One-hot encoding represents each category as a separate binary feature (or column). Each feature corresponds to a specific category and takes a value of 1 if the observation belongs to that category and 0 otherwise. It is suitable for nominal variables where there is no order or hierarchy among the categories.

We can use one-hot encoding for all categorical variables in our dataset. Let's see how we can do one-hot encoding in Python using the `OneHotEncoder` class from the `sklearn.preprocessing` module.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
area = ohe.fit_transform(df[["area"]])
display(area)
display(area.toarray())

`OneHotEncoder` returns a sparse matrix. A sparse matrix is a matrix that contains mostly zeros. Sparse matrices are useful because they save memory. However, they are not very easy to read.

To visualize a sparse matrix, we can convert it into a regular matrix using the `toarray` method.

The sparse format is useful when dealing with large datasets that contain a lot of zeros. For our dataset, the sparse format is not necessary.

### Task: encode all the categorical variables in our dataset using one-hot encoding.

In [ ]:
categorical_variables = ["gender", "dependents", "married", "education", "self_employed", "area"]
onehot_variables = ohe.fit_transform(df[categorical_variables])

The function `get_dummies` from pandas can also be used to do one-hot encoding. It returns a regular matrix.

In [ ]:
pd.get_dummies(df["area"])

It can also be used to do one-hot encoding of the entire dataset at once.

In [ ]:
categorical_variables = ["gender", "dependents", "married", "education", "self_employed", "area"]
pd.get_dummies(df, columns=categorical_variables)

## Binary Encoding

Do we need to encode binary variables? For instance, the column `self_employed` has two possible values: `Yes` and `No`. Can we just use them as they are?

The answer is yes*. We can use binary variables as they are, even if they are strings. However, we can also encode them using one-hot encoding. The advantage of one-hot encoding is that it makes it easier to add new categories in the future. For example, if we want to add a new category `Maybe` to the column `self_employed`, we can just add a new column `[0, 0]` to the one-hot encoded matrix.

\* _Actually, it depends. It works for most models in scikit-learn, because sklearn converts binary variables into numbers automatically. But it's not always the case, so it's better to encode binary variables as well to be safe._


In [ ]:
pd.get_dummies(df, columns=["gender"])

## Combining Numerical and Categorical Variables

Once we encode all the categorical variables, we can combine them with the numerical variables to create a single matrix. This matrix will be our feature matrix.

Let's see how to do that in Python. Let's use `pd.get_dummies` because it is much easier to use than `OneHotEncoder`.

In [ ]:
df_encoded = pd.get_dummies(df, columns=categorical_variables)
X = df_encoded.drop("loan_given", axis=1).values
# We do not need to encode the target variable, but it makes evaluation easier later
y = df_encoded["loan_given"].apply(lambda x: 1 if x == "Y" else 0).values
print(X.shape)
display(df_encoded.head())

### Task: train a decision tree classifier to predict whether a loan will be given to an applicant. Split the data into training and test sets. Use a classification report to evaluate the model.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import tree

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = tree.DecisionTreeClassifier()

model.fit(X_train, y_train)
print(classification_report(y_test, model.predict(X_test)))

# Cross-Validation

In the previous tutorial, we split the data into training and test sets. We trained the model on the training set and evaluated it on the test set. This approach is called **holdout validation**.

Holdout validation is a good approach, but it has one major drawback: it only uses part of the data for training. In our case, we used 80% of the data for training and 20% for testing. This means that we are not using 20% of the data for training. Moreover, the random split might have put most of the difficult examples in the training set and most of the easy examples in the test set. This would make the model look better than it actually is.

To solve this problem, we can use **cross-validation**. Cross-validation is a technique that allows us to use all the data for training and testing. It also allows us to evaluate the model on multiple test sets instead of just one test set.

There are many types of cross-validation. In this tutorial, we will use **k-fold cross-validation**. In k-fold cross-validation, we split the data into `k` folds (also called splits). Then, we train the model `k` times. Each time, we use a different fold for testing and the remaining folds for training. Finally, we average the results of the k models to get the final result.

10-fold and 5-fold cross-validation are the most common types of cross-validation. Let's see how to do it in Python using the `cross_validate` function from the `sklearn.model_selection` module.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classifier
clf = DecisionTreeClassifier()
# Define the scoring metrics
metrics = ["accuracy", "precision", "recall", "f1"]
# Perform 5-fold cross-validation
cv_scores = cross_validate(clf, X, y, cv=5, scoring=metrics)
# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)

`cross_validate` takes as input a model, a feature matrix, a target vector, the number of folds, and, optionally, a scoring function (metrics to evaluate the model). It returns a dictionary containing the scores of the model on each fold.

Have a look at the dictionary returned by `cross_validate` to understand what is in there.

The most important things to look at are the mean and the standard deviation of the scores. The mean tells us how good the model is on average. The standard deviation tells us how consistent the model is. If the standard deviation is high, it means that the model performs very differently on different folds.

In [ ]:
for metric in metrics:
    metric_key = f"test_{metric}"
    print(f"Mean {metric} : {cv_scores[metric_key].mean():.3f}, std: {cv_scores[metric_key].std():.3f}")

## Getting the Models

By default, the `cross_validate` function only returns the scores of the model. If we want to get the models themselves, we can set the parameter `return_estimator` to `True`.

In this case, the returned dictionary will include a key `estimator` that contains the list of models -- one for each fold. 

We can use these models to make predictions on new data.

In [ ]:
cv_scores = cross_validate(clf, X, y, cv=5, scoring=metrics, return_estimator=True)
models = cv_scores["estimator"]
print(models)

## Getting the Model with the Best Score

We can also get the model with best performance according to a specific metric. 

Let's get the model with highest F1.

In [ ]:
import numpy as np

# Get the scores for the specified metric
scores = cv_scores["test_f1"]
# Find the index of the model with the best performance
best_model_index = np.argmax(scores)
# Get the best model
best_model = cv_scores["estimator"][best_model_index]
print(best_model)

We can now use it to make predictions on new data!

In [ ]:
best_model.predict(X_test)

## Task: train a logistic regression model to predict whether a loan will be given to an applicant. Use 5-fold cross-validation to evaluate the model.

Use F1 score as the evaluation metric. Print the mean and standard deviation of the F1 scores. Also print the highest F1 score.

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
cv_scores = cross_validate(clf, X, y, cv=5, scoring="f1")
print("Cross-Validation Scores:", cv_scores)
print(f"Mean F1: {cv_scores['test_score'].mean():.3f}, std: {cv_scores['test_score'].std():.3f}, max: {cv_scores['test_score'].max():.3f}")

# Hyperparameter Tuning

So far, we have used the default hyperparameters of the models. Hyperparameters are parameters that are not learned during training. They are set before training and remain constant during training. For example, the maximum depth of a decision tree is a hyperparameter. Hyperparameters can have a big impact on the performance of the model.

Each model has its own set of hyperparameters. To check the hyperparameters of a model, we can check the documentation of corresponding sklearn class. For example, the documentation of the `DecisionTreeClassifier` class can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

We will learn how to optmise the hyperparameters of a model using grid search. Grid search is a technique that allows us to find the best combination of hyperparameters for a model. It works by trying all possible combinations and selecting the best one.

Let's see how to do that in Python using the `GridSearchCV` class from the `sklearn.model_selection` module.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the model
clf = DecisionTreeClassifier()
# Define the parameter grid
param_grid = {"max_depth": [1, 3, 5, 7, 9]}
# Perform grid search with cross-validation
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring="f1")
# Fit the grid search to the data
grid_search.fit(X, y)
# Print the best parameter and best score
print("Best Parameter:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


We first define the model we are going to optimise. Then, we define the hyperparameters we want to optimise. We put the hyperparameters in a dictionary. The keys of the dictionary are the names of the hyperparameters and the values are the values we want to try. In this case, we want to try 5 values for the maximum depth of the decision tree: 1, 3, 5, 7, and 9.

The `GridSearchCV` class takes as input the model, the hyperparameters, the number of folds, and the scoring function. Then, it performs cross-validation for each model considering all possible combinations of hyperparameters. 

We can then get the best hyperparameters using the `best_params_` attribute, the best score using `best_score_`, and the also best model using `best_estimator_`.

We can optimise multiple hyperparameters at the same time. Let's optimise the maximum depth and the minimum number of samples required to split an internal node.

In [ ]:
# Define the model
clf = DecisionTreeClassifier()
# Define the parameter grid
param_grid = {"max_depth": [1, 3, 5, 7, 9], "min_samples_leaf": [1, 5, 10]}
# Perform grid search with cross-validation
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring="f1")
# Fit the grid search to the data
grid_search.fit(X, y)
# Print the best parameter and best score
print("Best Parameter:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


In this case, we have 5 values for the maximum depth and 3 values for the minimum number of samples. This means that we will try 15 combinations of hyperparameters.

The best score did not change. This means that `min_samples_leaf` did not impact our model, and the default value is good enough.

Can we get a better score?

### Task: select at least one more hyperparameter and optimise it. Print the best hyperparameters and the best score.

Try to beat the current best score! (0.872)

In [ ]:
# Define the model
dt = DecisionTreeClassifier()
# Define the parameter grid
param_grid = {"max_depth": [1, 3, 5, 7, 9], "min_samples_leaf": [1, 5, 10], "min_samples_split": [2, 5, 10, 15], "max_features": ["sqrt", "log2"]}
# Perform grid search with cross-validation
grid_search_dt = GridSearchCV(dt, param_grid, cv=5, scoring="f1")
# Fit the grid search to the data
grid_search_dt.fit(X, y)
# Print the best parameter and best score
print("Best Parameter:", grid_search_dt.best_params_)
print("Best Score:", grid_search_dt.best_score_)


### Task: now optimise the hyperparameters of a logistic regression model. Print the best hyperparameters and the best score.

Can you outperform the decision tree classifier?

In [ ]:
# Define the model
logreg = LogisticRegression()
# Define the parameter grid
param_grid = {"C": [0.1, 1, 10, 100], "class_weight": ["balanced", None], "max_iter": [100, 200, 500]}
# Perform grid search with cross-validation
grid_search_logreg = GridSearchCV(logreg, param_grid, cv=5, scoring="f1")
# Fit the grid search to the data
grid_search_logreg.fit(X, y)
# Print the best parameter and best score
print("Best Parameter:", grid_search_logreg.best_params_)
print("Best Score:", grid_search_logreg.best_score_)


### Task: compare the best decision tree classifier and the best logistic regression model. Use a classification report and a confusion matrix to evaluate the models.

What are the main differences?

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

# Decision tree
print(classification_report(y, grid_search_dt.best_estimator_.predict(X)))
display(ConfusionMatrixDisplay.from_estimator(grid_search_dt.best_estimator_, X, y))
# Logisitc regression
print(classification_report(y, grid_search_logreg.best_estimator_.predict(X)))
display(ConfusionMatrixDisplay.from_estimator(grid_search_logreg.best_estimator_, X, y))

There are other ways of optimising hyperparameters. In sklearn, you can also use [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). Have a look at the documentation to learn more about them. [This tutorial](https://scikit-learn.org/stable/modules/grid_search.html) provides a nice overview of the different methods.

# Feature Selection

We have just learned how to optimise the hyperparameters of a model. We can also optimise the features used to train model. This is called **feature selection**. Feature selection is the process of selecting the most important features for a model. It is useful because it can reduce complexity and improve performance.

There are [many methods](https://scikit-learn.org/stable/modules/feature_selection.html) for feature selection. In this tutorial, we will learn about `SelectKBest`. `SelectKBest` is a method that selects the `k` best features according to a specific metric.

Let's see how it works in sklearn.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Number of features to select
k = 5
selector = SelectKBest(k=k, score_func=chi2)
X_selected = selector.fit_transform(X, y)
# Get the selected feature indices
feature_indices = selector.get_support(indices=True)
# Print the selected features
selected_features = df_encoded.drop("loan_given", axis=1).columns[feature_indices]
print("Selected Features:", selected_features)

In [ ]:
selector.scores_

We are using `chi2` as the scoring function. `chi2` (chi-squared) is a statistical test that measures the dependence between two variables. In this case, it measures the dependence between each feature and the target. The higher the score, the more important the feature.

We can check the scores of each feature using the `scores_` attribute.

In [ ]:
print(selector.scores_)

### Task: select the 5 best features and train a decision tree classifier using these features. Print the classification report and the confusion matrix.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classifier
clf = DecisionTreeClassifier()
# Define the scoring metrics
metrics = ["accuracy", "precision", "recall", "f1"]
# Perform 5-fold cross-validation
cv_scores = cross_validate(clf, X_selected, y, cv=5, scoring=metrics)
# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print(f"Mean F1: : {cv_scores['test_f1'].mean()}, std: {cv_scores['test_f1'].std()}")

### Task: do the same thing with a logistic regression model. Which model performs better with fewer features?

In [ ]:
clf = LogisticRegression()
# Define the scoring metrics
metrics = ["accuracy", "precision", "recall", "f1"]
# Perform 5-fold cross-validation
cv_scores = cross_validate(clf, X_selected, y, cv=5, scoring=metrics)
# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print(f"Mean F1: : {cv_scores['test_f1'].mean()}, std: {cv_scores['test_f1'].std()}")